In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly_express as px

event = pd.read_csv('event.csv')
distribution_center = pd.read_csv('distribution_center.csv')
inventory_items = pd.read_csv('inventory_items.csv')
order_items = pd.read_csv('order_items.csv')
orders = pd.read_csv('orders.csv')
users = pd.read_csv('users.csv', low_memory=False)
products = pd.read_csv('products.csv')

In [2]:
order_items.created_at = pd.to_datetime(order_items.created_at)

order_items = order_items.sort_values(['user_id', 'created_at'], ascending=[True, True])

order_items.loc[:, 'previous_purchase'] = order_items.groupby('user_id')['created_at'].shift(1)  # 이전 구매 날짜
order_items.loc[:, 'purchase_interval'] = (order_items['created_at'] - order_items['previous_purchase']).dt.days  # 구매 간격(일 단위)

In [3]:
snapshot_date = order_items['created_at'].max() + pd.Timedelta(days=1)

rfm = order_items.groupby('user_id').agg({
    'created_at': lambda x: (snapshot_date - x.max()).days,
    'user_id' : 'count',
    'sale_price' : 'sum'
}).rename(columns={
    'created_at' : 'Recency',
    'user_id' : 'Frequency',
    'sale_price' : 'Monetary'
}).reset_index()

In [4]:
rfm['R_score'] = pd.qcut(rfm['Recency'], 3, labels=[3, 2, 1])
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 3, labels=[1, 2, 3])
rfm['M_score'] = pd.qcut(rfm['Monetary'], 3, labels=[1, 2, 3])

rfm['RFM_score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)

In [5]:
def segment(x):
    if x['R_score'] == 3 and x['F_score'] == 3 and x['M_score'] == 3:
        return '우수 고객'
    elif x['R_score'] == 1 and x['F_score'] == 1 and x['M_score'] == 1:
        return '이탈 위험 고객'
    else:
        return '일반 고객'
    
rfm['Segment'] = rfm.apply(segment, axis=1)

In [6]:
# nan 값은 첫 구매
orders.created_at = pd.to_datetime(order_items.created_at)

orders = orders.sort_values(['user_id', 'created_at'], ascending=[True, True])

orders.loc[:, 'previous_purchase'] = orders.groupby('user_id')['created_at'].shift(1)  # 이전 구매 날짜
orders.loc[:, 'purchase_interval'] = (orders['created_at'] - orders['previous_purchase']).dt.days  # 구매 간격(일 단위)

order_segment = pd.merge(orders, rfm[['user_id', 'Segment']], how='left', on='user_id')
order_segment.head()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item,previous_purchase,purchase_interval,Segment
0,2,1,Cancelled,F,2023-07-26 13:22:36,NaN,NaN,NaN,1,NaT,NaN,우수 고객
1,3,1,Shipped,F,2023-11-04 02:25:04,NaN,2025-04-05 09:48:00 UTC,NaN,1,2023-07-26 13:22:36,100.0,우수 고객
2,1,1,Cancelled,F,2024-11-16 23:36:48,NaN,NaN,NaN,1,2023-11-04 02:25:04,378.0,우수 고객
3,4,2,Shipped,M,2022-10-28 12:19:58,NaN,2023-12-07 22:01:00 UTC,NaN,1,NaT,NaN,일반 고객
4,8,3,Processing,M,2022-06-21 02:11:38,NaN,NaN,NaN,1,NaT,NaN,일반 고객


In [7]:
# 세그먼트별 선호 제품이나 재구매 간격

segment_cnt = order_segment.groupby('user_id').Segment.unique().to_frame('Segment').reset_index()
segment_cnt.Segment = segment_cnt.Segment.apply(lambda x: x[0])
segment_cnt = segment_cnt.groupby('Segment').size().to_frame('cnt').reset_index().sort_values('cnt', ascending=False)

fig = px.bar(segment_cnt, x='Segment', y='cnt', color='Segment', text_auto=True)
fig.update_layout(title=dict(text='세그먼트별 고객 수',
                             x=0.5),
                xaxis_title='세그먼트',
                yaxis_title='고객 수',
                template='simple_white')
fig.show()

In [8]:
order_items_segment = pd.merge(order_items, rfm[['user_id', 'Segment']], how='left', on='user_id' )
user_segment = pd.merge(users, rfm[['user_id', 'Segment']], how='left', on='user_id')
event_segment = pd.merge(event, rfm[['user_id', 'Segment']], how='left', on='user_id')

In [9]:
order_items_segment_product = pd.merge(order_items_segment, products[['product_id', 'category', 'name', 'brand']], how='left', on='product_id')

In [10]:
# 비회원x 구매 경험

event = event[(event.user_id != -1) & (event.user_id.isin(event[event.event_type=='purchase'].user_id))]

event_cnt = event.groupby(['user_id', 'event_type']).size().to_frame('event_cnt').reset_index()

def is_repurchase(x):
    if x>=2:
        return 'repurchase'
    else:
        return 'no_repurchase'
    
event_cnt['is_repurchase'] = event_cnt.event_cnt.apply(is_repurchase)
event_cnt_df = pd.merge(event, event_cnt[['user_id', 'event_cnt', 'is_repurchase']], how='left', on='user_id')

In [11]:
orders = orders[~orders.status.isin(['Cancelled', 'Returned'])]

orders.sort_values(by=['user_id', 'created_at'], ascending=[True, True])

orders.loc[:, 'previous_purchase'] = orders.groupby('user_id')['created_at'].shift(1)  # 이전 구매 날짜
orders.loc[:, 'purchase_interval'] = (orders['created_at'] - orders['previous_purchase']).dt.days  # 구매 간격(일 단위)

orders_repurchase = pd.merge(orders, event_cnt[['user_id', 'is_repurchase']], how='left', on='user_id')

In [12]:
# 2 주문 수

event = event[event.user_id != -1]
event['created_at'] = pd.to_datetime(event['created_at'])

event = event.sort_values(by=['user_id', 'created_at'])

# 이전 방문 시간 (세션 단위로)
event['previous_visit'] = event.groupby('user_id')['created_at'].shift(1)

# 방문 간격 계산
event['visit_interval_days'] = (event['created_at'] - event['previous_visit']).dt.days
event['visit_interval_hours'] = (event['created_at'] - event['previous_visit']).dt.total_seconds() / 3600


In [13]:
# 전처리
event['created_at'] = pd.to_datetime(event['created_at'])

# user_id와 session_id로 그룹화해서 각 세션의 시작 시간 구하기
session_start = event.groupby(['user_id', 'session_id'])['created_at'].min().reset_index()
session_start = session_start.sort_values(by=['user_id', 'created_at'])

# 이전 세션 시간 추가
session_start['previous_session_time'] = session_start.groupby('user_id')['created_at'].shift(1)

# 재방문 간격 계산
session_start['revisit_interval'] = (session_start['created_at'] - session_start['previous_session_time']).dt.total_seconds() / 86400  # 시간 단위

In [14]:
session_start = session_start[session_start.revisit_interval.notna()]
session_start['revisit_interval'] = session_start.revisit_interval.round()

In [15]:
orders['is_repurchase'] = ~orders['previous_purchase'].isna()

In [16]:
order_items = order_items[order_items.status.isin(['Cancelled', 'Returned'])]
order_items = pd.merge(order_items, products[['product_id', 'brand']], how='left', on='product_id')
brand_cnt = order_items.groupby('user_id').brand.size().to_frame('brand_cnt').reset_index()

orders = pd.merge(orders, brand_cnt, how='left', on='user_id')

In [17]:
event_cnt = event.groupby('user_id').size().to_frame('event_cnt').reset_index()

orders = pd.merge(orders, event_cnt, how='left', on='user_id')

In [18]:
revisit = session_start.groupby('user_id').revisit_interval.mean().to_frame('revisit_interval').reset_index()

orders = pd.merge(orders, revisit, how='left', on='user_id')

In [19]:
event.groupby('user_id').size().median()

10.0

In [20]:
orders.revisit_interval.median()

98.0

### 유저 그룹 분할(직접 만든 기준)

| 유형                             | 조건                                                                 | 고객 유형 이름         |
|----------------------------------|----------------------------------------------------------------------|------------------------|
| 재구매o + 적극탐색 + 빠른 재방문 | 재구매o(2개 이상) + 이벤트 많음(10개 이상) + 재방문 주기 빠름(98일 이내) | 충성 고객 - 지연       |
| 재구매x + 탐색 적음 + 재방문 기간 긺 | 재구매x + (이벤트 적음 or 재방문 기간 긺(98일 이외))                       | 이탈 위험 - 준우       |
| 재구매x + 탐색 많음              | 재구매x + 이벤트 많음                                                 | 재구매 기회 존재 - 덕진 |
| 단일 브랜드 + 재방문 기간 긺     | 특정 브랜드 비중 매우 높음 + 재방문 기간 긺                              | 브랜드 의존 고객 - 태윤 |


In [21]:
def classify_user(row):
    if row['is_repurchase'] == 1 and row['event_cnt'] >= 10 and row['revisit_interval'] <= 98:
        return '충성 고객'
    elif row['is_repurchase'] == 0 and (row['event_cnt'] < 10 or row['revisit_interval'] > 98):
        return '이탈 위험'
    elif row['is_repurchase'] == 0 and row['event_cnt'] >= 10:
        return '재구매 기회 존재'
    elif row['brand_cnt'] == 1 and row['revisit_interval'] >= 98:         
        return '브랜드 의존 고객'
    else:
        return '기타'

orders['user_segment'] = orders.apply(classify_user, axis=1)
orders.drop_duplicates(subset='user_id', keep='last')[['user_id', 'user_segment']].groupby('user_segment').size()

seg = orders.drop_duplicates(subset='user_id', keep='last')[['user_id', 'user_segment']]

orders = orders.drop('user_segment', axis=1)

In [22]:
orders = pd.merge(orders, seg, how='left', on='user_id')

In [23]:
orders.drop_duplicates(subset='user_id', keep='last')[['user_id', 'user_segment']].groupby('user_segment').size()

user_segment
기타           10261
브랜드 의존 고객     1862
이탈 위험        31381
재구매 기회 존재    14714
충성 고객         8118
dtype: int64

In [24]:
purchase_cnt = orders.groupby('user_id').size().to_frame('purchase_cnt').reset_index()
orders = pd.merge(orders, purchase_cnt, how='left', on='user_id')
brand_user = orders[orders.user_segment == '브랜드 의존 고객']
non_brand_user = orders[orders.user_segment != '브랜드 의존 고객']

In [25]:
df = pd.DataFrame({'고객 구분':['단일 브랜드 이용', '다양한 브랜드 이용'],
                   '평균 구매 횟수':[brand_user.purchase_cnt.mean(), non_brand_user.purchase_cnt.mean()],
                   '평균 이벤트 발생 횟수':[brand_user.event_cnt.mean(), non_brand_user.event_cnt.mean()],
                   '평균 재방문 주기':[brand_user.revisit_interval.mean(), non_brand_user.revisit_interval.mean()]
                   })
fig = go.Figure()

fig.add_trace(go.Bar(x=['단일 브랜드 이용', '다양한 브랜드 이용'],
                     y=df['평균 구매 횟수'],
                     name='평균 구매 횟수',
                     text=df['평균 구매 횟수'].round(2),
                     textposition='auto',
                     marker_color=['#636EFA', '#EF553B']
                     
            ))

fig.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'args': [
                        {'y': [df['평균 구매 횟수']],
                         'text': [df['평균 구매 횟수'].round(2)],
                         'marker.color': [['#636EFA', '#EF553B']]},
                        {'title': '평균 구매 횟수'}
                    ],
                    'label': '평균 구매 횟수',
                    'method': 'update'
                },
                {
                    'args': [
                        {'y': [df['평균 이벤트 발생 횟수']],
                        'text': [df['평균 이벤트 발생 횟수'].round(2)],
                        'marker.color': [['#636EFA', '#EF553B']]},
                        {'title': '평균 이벤트 발생 횟수'}
                    ],
                    'label': '평균 이벤트 발생 횟수',
                    'method': 'update'
                },
                {
                    'args': [
                        {'y': [df['평균 재방문 주기']],
                        'text': [df['평균 재방문 주기'].round(2)],
                        'marker.color': [['#636EFA', '#EF553B']]},
                        {'title': '평균 재방문 주기'}
                    ],
                    'label': '평균 재방문 주기',
                    'method': 'update'
                }
            ],
            'direction': 'down',
            'showactive': True,
            'active': 0,
            'y': 1.15
        }
    ],
    title = dict(text='단일 브랜드 vs 다양한 브랜드 이용 고객 비교',
                 x=0.5),
    xaxis_title='고객 유형',
    yaxis_title='값',
    template='simple_white'
)
fig.show()

In [27]:
a= brand_user[['revisit_interval', 'purchase_cnt']]
a.corr()

fig = px.histogram(a, x='revisit_interval')
fig.update_layout(title=dict(text='재방문 기간에 따른 히스토그램',
                             x=0.5),
                xaxis_title='재방문 기간',
                yaxis_title='cnt')
fig.show()